In [ ]:
import wrf
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as colors
import cartopy
import xarray as xr
import pandas as pd
from ipywidgets import widgets, interactive
from matplotlib.widgets import Cursor

import holoviews as hv
import bokeh

hv.notebook_extension('bokeh')

In [ ]:
class AnnotatedCursor(Cursor):
    """
    A crosshair cursor like `~matplotlib.widgets.Cursor` with a text showing \
    the current coordinates.

    For the cursor to remain responsive you must keep a reference to it.
    The data of the axis specified as *dataaxis* must be in ascending
    order. Otherwise, the `numpy.searchsorted` call might fail and the text
    disappears. You can satisfy the requirement by sorting the data you plot.
    Usually the data is already sorted (if it was created e.g. using
    `numpy.linspace`), but e.g. scatter plots might cause this problem.
    The cursor sticks to the plotted line.

    Parameters
    ----------
    line : `matplotlib.lines.Line2D`
        The plot line from which the data coordinates are displayed.

    numberformat : `python format string <https://docs.python.org/3/\
    library/string.html#formatstrings>`_, optional, default: "{0:.4g};{1:.4g}"
        The displayed text is created by calling *format()* on this string
        with the two coordinates.

    offset : (float, float) default: (5, 5)
        The offset in display (pixel) coordinates of the text position
        relative to the cross hair.

    dataaxis : {"x", "y"}, optional, default: "x"
        If "x" is specified, the vertical cursor line sticks to the mouse
        pointer. The horizontal cursor line sticks to *line*
        at that x value. The text shows the data coordinates of *line*
        at the pointed x value. If you specify "y", it works in the opposite
        manner. But: For the "y" value, where the mouse points to, there might
        be multiple matching x values, if the plotted function is not biunique.
        Cursor and text coordinate will always refer to only one x value.
        So if you use the parameter value "y", ensure that your function is
        biunique.

    Other Parameters
    ----------------
    textprops : `matplotlib.text` properties as dictionary
        Specifies the appearance of the rendered text object.

    **cursorargs : `matplotlib.widgets.Cursor` properties
        Arguments passed to the internal `~matplotlib.widgets.Cursor` instance.
        The `matplotlib.axes.Axes` argument is mandatory! The parameter
        *useblit* can be set to *True* in order to achieve faster rendering.

    """

    def __init__(self, line, numberformat="{0:.4g};{1:.4g}", offset=(5, 5),
                 dataaxis='x', textprops={}, **cursorargs):
        # The line object, for which the coordinates are displayed
        self.line = line
        # The format string, on which .format() is called for creating the text
        self.numberformat = numberformat
        # Text position offset
        self.offset = np.array(offset)
        # The axis in which the cursor position is looked up
        self.dataaxis = dataaxis

        # First call baseclass constructor.
        # Draws cursor and remembers background for blitting.
        # Saves ax as class attribute.
        super().__init__(**cursorargs)

        # Default value for position of text.
        self.set_position(self.line.get_xdata()[0], self.line.get_ydata()[0])
        # Create invisible animated text
        self.text = self.ax.text(
            self.ax.get_xbound()[0],
            self.ax.get_ybound()[0],
            "0, 0",
            animated=bool(self.useblit),
            visible=False, **textprops)
        # The position at which the cursor was last drawn
        self.lastdrawnplotpoint = None

    def onmove(self, event):
        """
        Overridden draw callback for cursor. Called when moving the mouse.
        """

        # Leave method under the same conditions as in overridden method
        if self.ignore(event):
            self.lastdrawnplotpoint = None
            return
        if not self.canvas.widgetlock.available(self):
            self.lastdrawnplotpoint = None
            return

        # If the mouse left drawable area, we now make the text invisible.
        # Baseclass will redraw complete canvas after, which makes both text
        # and cursor disappear.
        if event.inaxes != self.ax:
            self.lastdrawnplotpoint = None
            self.text.set_visible(False)
            super().onmove(event)
            return

        # Get the coordinates, which should be displayed as text,
        # if the event coordinates are valid.
        plotpoint = None
        if event.xdata is not None and event.ydata is not None:
            # Get plot point related to current x position.
            # These coordinates are displayed in text.
            plotpoint = self.set_position(event.xdata, event.ydata)
            # Modify event, such that the cursor is displayed on the
            # plotted line, not at the mouse pointer,
            # if the returned plot point is valid
            if plotpoint is not None:
                event.xdata = plotpoint[0]
                event.ydata = plotpoint[1]

        # If the plotpoint is given, compare to last drawn plotpoint and
        # return if they are the same.
        # Skip even the call of the base class, because this would restore the
        # background, draw the cursor lines and would leave us the job to
        # re-draw the text.
        if plotpoint is not None and plotpoint == self.lastdrawnplotpoint:
            return

        # Baseclass redraws canvas and cursor. Due to blitting,
        # the added text is removed in this call, because the
        # background is redrawn.
        super().onmove(event)

        # Check if the display of text is still necessary.
        # If not, just return.
        # This behaviour is also cloned from the base class.
        if not self.get_active() or not self.visible:
            return

        # Draw the widget, if event coordinates are valid.
        if plotpoint is not None:
            # Update position and displayed text.
            # Position: Where the event occurred.
            # Text: Determined by set_position() method earlier
            # Position is transformed to pixel coordinates,
            # an offset is added there and this is transformed back.
            temp = [event.xdata, event.ydata]
            temp = self.ax.transData.transform(temp)
            temp = temp + self.offset
            temp = self.ax.transData.inverted().transform(temp)
            self.text.set_position(temp)
            self.text.set_text(self.numberformat.format(*plotpoint))
            self.text.set_visible(self.visible)

            # Tell base class, that we have drawn something.
            # Baseclass needs to know, that it needs to restore a clean
            # background, if the cursor leaves our figure context.
            self.needclear = True

            # Remember the recently drawn cursor position, so events for the
            # same position (mouse moves slightly between two plot points)
            # can be skipped
            self.lastdrawnplotpoint = plotpoint
        # otherwise, make text invisible
        else:
            self.text.set_visible(False)

        # Draw changes. Cannot use _update method of baseclass,
        # because it would first restore the background, which
        # is done already and is not necessary.
        if self.useblit:
            self.ax.draw_artist(self.text)
            self.canvas.blit(self.ax.bbox)
        else:
            # If blitting is deactivated, the overridden _update call made
            # by the base class immediately returned.
            # We still have to draw the changes.
            self.canvas.draw_idle()

    def set_position(self, xpos, ypos):
        """
        Finds the coordinates, which have to be shown in text.

        The behaviour depends on the *dataaxis* attribute. Function looks
        up the matching plot coordinate for the given mouse position.

        Parameters
        ----------
        xpos : float
            The current x position of the cursor in data coordinates.
            Important if *dataaxis* is set to 'x'.
        ypos : float
            The current y position of the cursor in data coordinates.
            Important if *dataaxis* is set to 'y'.

        Returns
        -------
        ret : {2D array-like, None}
            The coordinates which should be displayed.
            *None* is the fallback value.
        """

        # Get plot line data
        xdata = self.line.get_xdata()
        ydata = self.line.get_ydata()

        # The dataaxis attribute decides, in which axis we look up which cursor
        # coordinate.
        if self.dataaxis == 'x':
            pos = xpos
            data = xdata
            lim = self.ax.get_xlim()
        elif self.dataaxis == 'y':
            pos = ypos
            data = ydata
            lim = self.ax.get_ylim()
        else:
            raise ValueError(f"The data axis specifier {self.dataaxis} should "
                             f"be 'x' or 'y'")

        # If position is valid and in valid plot data range.
        if pos is not None and lim[0] <= pos <= lim[-1]:
            # Find closest x value in sorted x vector.
            # This requires the plotted data to be sorted.
            index = np.searchsorted(data, pos)
            # Return none, if this index is out of range.
            if index < 0 or index >= len(data):
                return None
            # Return plot point as tuple.
            return (xdata[index], ydata[index])

        # Return none if there is no good related point for this x position.
        return None

    def clear(self, event):
        """
        Overridden clear callback for cursor, called before drawing the figure.
        """

        # The base class saves the clean background for blitting.
        # Text and cursor are invisible,
        # until the first mouse move event occurs.
        super().clear(event)
        if self.ignore(event):
            return
        self.text.set_visible(False)

    def _update(self):
        """
        Overridden method for either blitting or drawing the widget canvas.

        Passes call to base class if blitting is activated, only.
        In other cases, one draw_idle call is enough, which is placed
        explicitly in this class (see *onmove()*).
        In that case, `~matplotlib.widgets.Cursor` is not supposed to draw
        something using this method.
        """

        if self.useblit:
            super()._update()

In [ ]:
input = Dataset("/data/keeling/a/jcurtis2/d/wrf_partmc_keeling_dev/wrf_partmc/mozart_to_wrf/wrfinput_d01")
temp = wrf.getvar(input, "T")
lats, lons = wrf.latlon_coords(temp)
lats = wrf.to_np(lats)
lons = wrf.to_np(lons)

In [ ]:
dir = '/data/keeling/a/jcurtis2/a/wrf_partmc/emissions'

In [ ]:
sources = np.loadtxt('source_names.dat',dtype='str')
source_names = sources[:,0]

In [ ]:
file = 'emission_statistics.nc'
data = Dataset('%s/%s' %(dir,file), "r", format="NETCDF4")

#df=pd.read_csv('source_names.dat',sep=',',header=[0])
#input = Dataset("emission_statistics.nc")
#print(df.values[0])
vars = list(data.variables.keys())

matches = [vars for vars in vars if "source_" in vars]

matches_ts = [vars for vars in vars if "emission_" in vars]
data_frame = [[0,1,2,3,4]]
data_frame_timeseries = [list(range(49))]
data_frame_map = []
labels = ["x"]
for i_source in range(0,len(matches)-1,2):
    data_frame.append(data.variables[matches[i_source]][:])
    labels.append("%s" % (source_names[int(matches[i_source][-2:])-1]))
    data_frame_timeseries.append(data.variables[matches_ts[i_source]][:])
    data_frame_map.append(data.variables['emission_values_%s' % matches[i_source][-2:]][:].flatten())
data.close()

In [ ]:
df2 = pd.DataFrame(np.array(data_frame).T, columns=labels)
df_timeseries = pd.DataFrame(np.array(data_frame_timeseries).T, columns=labels)
df_map = pd.DataFrame(np.array(data_frame_map).T, columns=labels[1:])

In [ ]:
def plotit(emission_source):
    """
    

    Args:
    -----


    Returns:
    --------
    

    """
    fig = plt.figure(figsize=(12, 3))
#    fig = figure()
    gs = fig.add_gridspec(1, 3,  width_ratios=(4, 7,7),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.25, hspace=0.05)

    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[0, 2],projection=cartopy.crs.PlateCarree())
    spec_colors = ['yellow','red','blue','green','black']
    ax1.pie(df2[emission_source],colors=spec_colors,labels=["OIN","SO4","NO3","OC","EC"],
           labeldistance=None)
    ax1.legend(loc='upper center',bbox_to_anchor=(.5,0),ncol=3)
    #ax1.scatter([0,1,2,3,4],df2[emission_source],s=None,c=spec_colors)
#    ax1.set_xticks([0,1,2,3,4],["OIN","SO4","NO3","OC","EC"])
#    ax1.set_ylim([0,1])
#    ax1.set_ylabel('Mass fraction')
    time = np.arange(0,25)
    line, = ax2.plot(time,df_timeseries[emission_source][7:7+25])
    
    cursor = AnnotatedCursor(
        line=line,
        numberformat="{0:.2f}\n{1:.2f}",
        dataaxis='x', offset=[0, 0],
        textprops={'color': 'blue', 'fontweight': 'bold'},
        ax=ax2,
        useblit=True,
        color='red',
        linewidth=2)
    ylims = ax2.get_ylim()
    ax2.set_ylim([0,1.1*df_timeseries[emission_source].max()])
    ax2.set_xlim([0,24])
    ax2.set_xlabel('Local time')
    ax2.set_ylabel('emission strength')
    ax2.set_xticks([0,6,12,18,24])
    ax2.set_yticks([])

    maxval = np.array(df_map[emission_source]).max()
    ax3.pcolormesh(lons[:,:],lats[:,:],(np.array(df_map[emission_source]).reshape(159,169)),norm=colors.LogNorm(maxval/1e3,maxval),
                  transform=cartopy.crs.PlateCarree())
    

    ax3.coastlines(resolution='50m', color='black', linewidth=1)
    states_provinces = cartopy.feature.NaturalEarthFeature(
            category='cultural',
            name='admin_1_states_provinces_lines',
            scale='50m',
            facecolor='none')
    ax3.add_feature(states_provinces, edgecolor='black')
    #r = ax2.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)
    plt.show()

In [ ]:
# Create two bounded text box that allow only numbers between the min year (2012) and the max year (2015)
from ipywidgets import Layout

layout = widgets.Layout(width='400px', height='30px')
# Make a dropdown to select the Area, or "All"
source = widgets.Dropdown(
    options=labels[1:],
    value=labels[25],
    description='Emission Source',
    layout = layout,
    style= {'description_width': 'initial'}
)

In [ ]:
interactive(plotit, emission_source=source)